# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization, Metadata
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired, trim_single
from qiime2.plugins.demux.methods import filter_samples

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'parameter-set-01.yaml')
experiment_name = 'exp-01'
base_dir = os.path.join('/', 'home', 'username', 'pipeline-dir')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'manifest.csv')
replace_files = False
trim = None
metadata_file = None
threads = 1
demux_file = None

In [3]:
# Parameters
base_dir = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri"
class_col = "group-id"
classifier_file = "/mnt/nupeb/rede-micro/datasets/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
experiment_name = "ana-flavia-HSD-NCxHSD-NR"
manifest_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-HSD-NCxHSD-NR.csv"
metadata_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-HSD-NCxHSD-NR.tsv"
overlap = 12
phred = 20
replace_files = False
threads = 6
top_n = 20
trim = {"forward_primer": "CCTACGGGRSGCAGCAG", "overlap": 8, "reverse_primer": "GGACTACHVGGGTWTCTAAT"}
trunc_f = 0
trunc_r = 0


In [4]:
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))
data_dir =  os.path.abspath(os.path.join(base_dir, 'data'))
raw_data_dir =  os.path.abspath(os.path.join(data_dir, 'raw'))
interim_data_dir =  os.path.abspath(os.path.join(data_dir, 'interim'))

### Defining names and paths

In [5]:
# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

New qiime-artifacts folder path created: /mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-HSD-NCxHSD-NR/qiime-artifacts
New img folder path created: /mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-HSD-NCxHSD-NR/imgs


In [6]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
    direction = 'single'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
    direction = 'paired'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [7]:
sufix = str(direction)
if trim and len(trim.keys()) == 3:
    sufix += f'-{trim}'
    
# Define the output artifact full path
if demux_file is None:
    demux_file = os.path.join(out_dir, f'demux-{direction}.qza')
demux_view = os.path.join(out_dir, f'demux-{direction}.qzv')
demux_file_trim = os.path.join(out_dir, f'demux-{direction}-trim.qza')
demux_view_trim = os.path.join(out_dir, f'demux-{direction}-trim.qzv')

## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file.

In [8]:
# If the metadata file is not defined, use the default metadata file
metadata_qa = Metadata.load(metadata_file)

In [9]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    interim_demux_path = os.path.join(interim_data_dir, f'demux-{direction}.qza')
    
    # Create new file path for interim dir
    if not os.path.isdir(interim_data_dir):
        !mkdir -p {interim_data_dir}
        
    # Load Artifact from interim folder
    if os.path.isfile(interim_demux_path):
        artifact = Artifact.load(interim_demux_path)
        
    # Create new Artifact using Manifest
    else:
        artifact = Artifact.import_data(d_type, manifest_file, view_type=v_type)
        if not os.path.isfile(interim_demux_path):
            artifact.save(interim_demux_path)
            
    # Filter and Save Artifact
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    artifact.save(demux_file)
    
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)
        
elif os.path.isfile(demux_file):
    
    # Load Artifact
    artifact = Artifact.load(demux_file)
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)

/opt/anaconda3/envs/qiime2-2023.7.1/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


/opt/anaconda3/envs/qiime2-2023.7.1/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


## Step report


In [10]:
print(demux_view_obj)

# Render Visualization
demux_view_obj

<visualization: Visualization uuid: f79940ab-98bd-4d59-8a5a-ee363cc34a21>


<visualization: Visualization uuid: f79940ab-98bd-4d59-8a5a-ee363cc34a21>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [11]:
demux_file_trim = os.path.join(out_dir, f'demux-{direction}-trim.qza')
demux_view_trim = os.path.join(out_dir, f'demux-{direction}-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = None
    
    if n_directions == 1:
        res = trim_single(
            demultiplexed_sequences=artifact,
            front=forward_primer,
            adapter=reverse_reverse_complement,
            cores=threads,
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    elif n_directions == 2:
        res = trim_paired(
            demultiplexed_sequences=artifact,
            front_f=forward_primer,
            front_r=reverse_primer,
            # adapter_f=reverse_reverse_complement,
            # adapter_r=forward_reverse_complement,
            cores=threads,
            overlap=trim['overlap'],
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    else:
        print(f'ERROR: invalid number of directions {n_directions}')

    if res is not None:
        res.save(demux_file_trim)
        Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-base 33 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-xd6mu4sm/S210421121698_50_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-xd6mu4sm/S210421121698_51_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2/lauro/data/52e2b061-359f-40a7-8b67-0387537111da/data/S210421121698_50_L001_R1_001.fastq.gz /tmp/qiime2/lauro/data/52e2b061-359f-40a7-8b67-0387537111da/data/S210421121698_51_L001_R2_001.fastq.gz

This is cutadapt 4.4 with Python 3.8.15
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-

Finished in 0.146 s (10.221 µs/read; 5.87 M reads/minute).

=== Summary ===

Total read pairs processed:             14,319
  Read 1 with adapter:                  13,866 (96.8%)
  Read 2 with adapter:                  12,833 (89.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            1,875 (13.1%)
Pairs written (passing filters):        12,444 (86.9%)

Total basepairs processed:     7,302,690 bp
  Read 1:     4,367,295 bp
  Read 2:     2,935,395 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):      5,801,125 bp (79.4%)
  Read 1:     3,540,895 bp
  Read 2:     2,260,230 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 13866 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
11	1	0.0	0	1
15	1	0.0	0	1
1

Finished in 0.199 s (9.505 µs/read; 6.31 M reads/minute).

=== Summary ===

Total read pairs processed:             20,959
  Read 1 with adapter:                  20,357 (97.1%)
  Read 2 with adapter:                  19,037 (90.8%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            2,450 (11.7%)
Pairs written (passing filters):        18,509 (88.3%)

Total basepairs processed:    10,689,090 bp
  Read 1:     6,392,495 bp
  Read 2:     4,296,595 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):      8,628,512 bp (80.7%)
  Read 1:     5,266,608 bp
  Read 2:     3,361,904 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 20357 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
15	1	0.0	0	1
16	1	0.0	0	1
17

Finished in 0.172 s (8.540 µs/read; 7.03 M reads/minute).

=== Summary ===

Total read pairs processed:             20,194
  Read 1 with adapter:                  19,610 (97.1%)
  Read 2 with adapter:                  18,045 (89.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            2,655 (13.1%)
Pairs written (passing filters):        17,539 (86.9%)

Total basepairs processed:    10,298,940 bp
  Read 1:     6,159,170 bp
  Read 2:     4,139,770 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):      8,176,314 bp (79.4%)
  Read 1:     4,990,682 bp
  Read 2:     3,185,632 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 19610 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
15	3	0.0	0	3
17	7	0.0	0	7
18

Finished in 0.313 s (4.645 µs/read; 12.92 M reads/minute).

=== Summary ===

Total read pairs processed:             67,293
  Read 1 with adapter:                  65,501 (97.3%)
  Read 2 with adapter:                  61,930 (92.0%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            6,987 (10.4%)
Pairs written (passing filters):        60,306 (89.6%)

Total basepairs processed:    34,319,430 bp
  Read 1:    20,524,365 bp
  Read 2:    13,795,065 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):     28,113,964 bp (81.9%)
  Read 1:    17,159,382 bp
  Read 2:    10,954,582 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 65501 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
11	1	0.0	0	1
12	1	0.0	0	1
1

...

Finished in 0.447 s (5.480 µs/read; 10.95 M reads/minute).

=== Summary ===

Total read pairs processed:             81,485
  Read 1 with adapter:                  79,303 (97.3%)
  Read 2 with adapter:                  74,804 (91.8%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            8,666 (10.6%)
Pairs written (passing filters):        72,819 (89.4%)

Total basepairs processed:    41,557,350 bp
  Read 1:    24,852,925 bp
  Read 2:    16,704,425 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):     33,947,823 bp (81.7%)
  Read 1:    20,720,493 bp
  Read 2:    13,227,330 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 79303 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
9	2	0.3	0	2
10	2	0.1	0	2
12

Finished in 0.200 s (7.339 µs/read; 8.18 M reads/minute).

=== Summary ===

Total read pairs processed:             27,196
  Read 1 with adapter:                  26,427 (97.2%)
  Read 2 with adapter:                  24,481 (90.0%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            3,391 (12.5%)
Pairs written (passing filters):        23,805 (87.5%)

Total basepairs processed:    13,869,960 bp
  Read 1:     8,294,780 bp
  Read 2:     5,575,180 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):     11,097,905 bp (80.0%)
  Read 1:     6,774,112 bp
  Read 2:     4,323,793 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 26427 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
13	1	0.0	0	1
14	2	0.0	0	2
17

Finished in 0.215 s (6.553 µs/read; 9.16 M reads/minute).

=== Summary ===

Total read pairs processed:             32,857
  Read 1 with adapter:                  31,953 (97.2%)
  Read 2 with adapter:                  29,876 (90.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            3,780 (11.5%)
Pairs written (passing filters):        29,077 (88.5%)

Total basepairs processed:    16,757,070 bp
  Read 1:    10,021,385 bp
  Read 2:     6,735,685 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):     13,555,322 bp (80.9%)
  Read 1:     8,273,880 bp
  Read 2:     5,281,442 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 31953 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
11	2	0.0	0	2
12	1	0.0	0	1
13

/opt/anaconda3/envs/qiime2-2023.7.1/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


/opt/anaconda3/envs/qiime2-2023.7.1/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)
